In [ ]:
import pandas as pd

Create combinations of sentences

In [ ]:
# download unter https://datanizing.com/llm-tag/arxiv-2023-llm-linguistic.parquet
df = pd.read_parquet("arxiv-2023-llm-linguistic.parquet", columns=['title', 'abstract', 'sentences'])

In [ ]:
df.columns

In [ ]:
from tqdm.auto import tqdm
import regex as re
embed_sentences = []
for j, r in tqdm(df.iterrows(), total=len(df)):
    sentences = r["sentences"]
    for i in range(0, len(sentences), 3):
        start = max(0, i)
        stop = min(start+5, len(sentences))
        text = re.sub(" +", " ", " ".join(sentences[start:stop]).replace("\n", " "))
        embed_sentences.append({ "id": j, "title": r["title"], 
                                 "text": text  })
        # don't continue if last sentence reached
        if stop == len(sentences):
            break

In [ ]:
len(embed_sentences)

In [ ]:
import json
with open("llm-abstract-sentences.json", "w") as f:
    f.write(json.dumps(embed_sentences))

Write JSON dataset for generative model

In [ ]:
import json
system_prompt = """You are an educated researcher and always answer in correct scientific terms. 
You are very deep into AI and its methodologies. You are very creative."""

with open("llm-abstract-dataset.jsonl", "w") as outfile:
    for i, r in df.iterrows():
        title = r["title"]
        abstract = r["abstract"]
        j = { "messages": [ { "content": system_prompt, "role": "system" },
                            { "content": f"Write an abstract with the title '{title}'", "role": "user" },
                            { "content": abstract, "role": "assistant" } ] }
        print(json.dumps(j), file=outfile)